# Imports

In [1]:
pip install datasets

In [2]:
!pip install PyDrive

In [3]:
from datasets import load_dataset
from sklearn.model_selection import train_test_split

In [4]:
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
import numpy as np
import torch
from transformers import BertTokenizer, BertModel
import torch.nn as nn
from transformers import BertModel, BertTokenizer
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import accuracy_score
from tqdm import tqdm
from datasets import load_dataset
from sklearn.model_selection import train_test_split
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import requests

In [5]:
urls = [
    "https://raw.githubusercontent.com/kzGarifullin/TDA-for-Transformers/main/scripts/directed_TDA_train.py",
    "https://raw.githubusercontent.com/kzGarifullin/TDA-for-Transformers/main/scripts/baseline_train.py"
]

for url in urls:
    response = requests.get(url)
    filename = url.split("/")[-1]
    with open(filename, 'wb') as file:
        file.write(response.content)

print("Files downloaded successfully.")


Files downloaded successfully.


In [6]:
from directed_TDA_train import directed_epoch_validation, Directed_classifier, train_directed
from baseline_train import baseline_epoch_validation, train_baseline, Classifier

# Dataset preparation

Let's download the dataset and the features already calculated on it.

In [7]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [8]:
file_id = '1-44fM5xBTMsgffUhj8aIKTOPj2EPlnD2' # URL id.
downloaded = drive.CreateFile({'id': file_id})
downloaded.GetContentFile('undirected_features_144.npy')

file_id = '1SXTTY7VZ77Gu5J4Z_RYIcC-Y2x8tKFhU' # URL id.
downloaded = drive.CreateFile({'id': file_id})
downloaded.GetContentFile('dataset_part1.pt')


# Colab download

If you have the dataset and features on your disk, then use this code to download them

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp /content/drive/MyDrive/undirected_features_144.npy  /content/
!cp /content/drive/MyDrive/dataset_part1.pt  /content/

# Dataset creation

In [9]:
import torch
from torch.utils.data import DataLoader

# Load the dataset part
dataset_part1 = torch.load('dataset_part1.pt')  # In one Colab account
# Create DataLoader
batch_size = 32
loader_part1 = DataLoader(dataset_part1, batch_size=batch_size, shuffle=False)

In [10]:
undirected_features = np.load('/content/undirected_features_144.npy')
undirected_features[undirected_features == np.inf] = 1000
print(undirected_features.shape)
undirected_features_without_nan = undirected_features[:, ~np.isnan(undirected_features).any(axis=0)]
print(undirected_features_without_nan.shape)

(12500, 2016)
(12500, 1880)


In [11]:
all_labels = []
all_texts = []

for batch in loader_part1:
    inputs, labels = batch
    all_labels.extend(labels.numpy())
    all_texts.extend(inputs.numpy())

all_labels = torch.tensor(all_labels)
all_texts = torch.tensor(all_texts)

<ipython-input-11-387134fe25d8>:10: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:274.)
  all_texts = torch.tensor(all_texts)


In [12]:
X_train, X_test, undir_feat_train, undir_feat_test, y_train, y_test = train_test_split(all_texts, undirected_features_without_nan, all_labels, test_size=0.2, random_state=42)
print(len(X_train))
print(len(X_test))
print(len(undir_feat_train))
print(len(undir_feat_test))
print(len(y_train))
print(len(y_test))

10000
2500
10000
2500
10000
2500


In [13]:
feat_train_tensor = torch.tensor(undir_feat_train, dtype=torch.float32)
feat_test_tensor = torch.tensor(undir_feat_test, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).unsqueeze(1)  # Adding a dimension for compatibility
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).unsqueeze(1)  # Adding a dimension for compatibility

feat_train_dataset = TensorDataset(feat_train_tensor, y_train_tensor)
feat_test_dataset = TensorDataset(feat_test_tensor, y_test_tensor)

feat_train_loader = DataLoader(feat_train_dataset, batch_size=batch_size, shuffle=False)
feat_test_loader = DataLoader(feat_test_dataset, batch_size=batch_size, shuffle=False)

<ipython-input-13-c0c43432f5f7>:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_train_tensor = torch.tensor(y_train, dtype=torch.float32).unsqueeze(1)  # Adding a dimension for compatibility
<ipython-input-13-c0c43432f5f7>:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_test_tensor = torch.tensor(y_test, dtype=torch.float32).unsqueeze(1)  # Adding a dimension for compatibility


In [14]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
max_length = 64  # Maximum sequence length for BERT
batch_size = 32

train_labels_tensor = torch.tensor(y_train)
val_labels_tensor = torch.tensor(y_test)

train_dataset = TensorDataset(X_train, train_labels_tensor)
val_dataset = TensorDataset(X_test, val_labels_tensor)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)
val_loader = DataLoader(val_dataset, batch_size=batch_size)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

<ipython-input-14-c06a342166da>:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_labels_tensor = torch.tensor(y_train)
<ipython-input-14-c06a342166da>:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  val_labels_tensor = torch.tensor(y_test)


#Baseline

Lets train baseline approach (without using pretrained features).

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_baseline = BertModel.from_pretrained('bert-base-uncased')
classifier_baseline = Classifier(input_size=768, num_classes=2)
classifier_baseline = classifier_baseline.to(device)
model_baseline = model_baseline.to(device)
criterion = nn.CrossEntropyLoss()
optimizer_baseline = torch.optim.Adam(classifier_baseline.parameters(), lr=1e-3)
num_epochs = 15
print("device:", device)
train_baseline(model_baseline, classifier_baseline, criterion, optimizer_baseline, train_loader, val_loader, num_epochs, device)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

device: cuda


100%|██████████| 313/313 [00:34<00:00,  9.10it/s]


Epoch 1/15, Validation Accuracy: 0.6012
Validation MCC: 0.2420145862356039
Train loss:  208.89637756347656
_______________________


100%|██████████| 313/313 [00:35<00:00,  8.77it/s]


...

100%|██████████| 313/313 [00:37<00:00,  8.30it/s]


Epoch 12/15, Validation Accuracy: 0.6756
Validation MCC: 0.3792954440394778
Train loss:  175.66885375976562
_______________________


100%|██████████| 313/313 [00:37<00:00,  8.28it/s]


Epoch 13/15, Validation Accuracy: 0.676
Validation MCC: 0.37973855882061847
Train loss:  175.08448791503906
_______________________


 85%|████████▍ | 265/313 [00:31<00:05,  8.27it/s]

In [ ]:
num_epochs = 5
print("device: ", device)
train_baseline(model_baseline, classifier_baseline, criterion, optimizer_baseline, train_loader, val_loader, num_epochs, device)

device:  cuda


100%|██████████| 313/313 [00:39<00:00,  7.98it/s]


Epoch 1/5, Validation Accuracy: 0.6776
Validation MCC: 0.3824881872765368
Train loss:  173.6617889404297
_______________________


100%|██████████| 313/313 [00:36<00:00,  8.46it/s]


Epoch 2/5, Validation Accuracy: 0.6784
Validation MCC: 0.38402222222422344
Train loss:  173.2695770263672
_______________________


...

Epoch 3/5, Validation Accuracy: 0.6788
Validation MCC: 0.38446952735550943
Train loss:  172.9084014892578
_______________________


100%|██████████| 313/313 [00:37<00:00,  8.31it/s]


Epoch 4/5, Validation Accuracy: 0.68
Validation MCC: 0.38676724362270454
Train loss:  172.5740966796875
_______________________


100%|██████████| 313/313 [00:37<00:00,  8.29it/s]


Epoch 5/5, Validation Accuracy: 0.6796
Validation MCC: 0.38568547183100765
Train loss:  172.2631378173828
_______________________


In [ ]:
num_epochs = 5
print("device: ", device)
train_baseline(model_baseline, classifier_baseline, criterion, optimizer_baseline, train_loader, val_loader, num_epochs, device)

device: cuda


100%|██████████| 313/313 [00:34<00:00,  8.97it/s]


Epoch 1/5, Validation Accuracy: 0.6804
Validation MCC: 0.38881993256569214
Train loss:  173.50083923339844
_______________________


100%|██████████| 313/313 [00:34<00:00,  8.97it/s]


Epoch 2/5, Validation Accuracy: 0.6808
Validation MCC: 0.38925841663844446
Train loss:  173.1157989501953
_______________________


...

Epoch 3/5, Validation Accuracy: 0.6812
Validation MCC: 0.38969948901623547
Train loss:  172.7610626220703
_______________________


100%|██████████| 313/313 [00:34<00:00,  8.98it/s]


Epoch 4/5, Validation Accuracy: 0.6804
Validation MCC: 0.3875323980308175
Train loss:  172.4326629638672
_______________________


100%|██████████| 313/313 [00:34<00:00,  8.98it/s]


Epoch 5/5, Validation Accuracy: 0.682
Validation MCC: 0.3909065733725074
Train loss:  172.1269989013672
_______________________


We achieved 68% accuracy!

# Undirected

Now let's concatenate our pre-calculated features with the output of the BERT and train the network.

In [16]:
import torch
import torch.nn as nn

class Nonlin_Directed_classifier(nn.Module):
    def __init__(self, input_size, hidden_size=512, num_classes=2):
        super(Nonlin_Directed_classifier, self).__init__()
        self.layer1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.layer2 = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        out = self.layer1(x)
        out = self.relu(out)
        out = self.layer2(out)
        return out

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
undir_classifier = Nonlin_Directed_classifier(input_size=2648,hidden_size=512,  num_classes=2)
undir_classifier = undir_classifier.to(device)
model_undir = BertModel.from_pretrained('bert-base-uncased')
model_undir = model_undir.to(device)
criterion = nn.CrossEntropyLoss()
optimizer_undir = torch.optim.Adam(undir_classifier.parameters(), lr=1e-3)
num_epochs = 15
print("device: ", device)
train_directed(model_undir, optimizer_undir, criterion, undir_classifier, train_loader, val_loader, feat_train_loader, feat_test_loader, num_epochs, device)


device:  cuda


313it [00:35,  8.92it/s]
79it [00:09,  8.63it/s]


Epoch 1/15, Validation Accuracy: 0.528
Validation MCC: 0.03324870478204374
Train loss:  799.1000366210938
_______________________


313it [00:35,  8.89it/s]
79it [00:08,  9.10it/s]


Epoch 2/15, Validation Accuracy: 0.5468
Validation MCC: 0.08568883874780396
Train loss:  216.45327758789062
_______________________


...

Epoch 9/15, Validation Accuracy: 0.5224
Validation MCC: -0.009372507609563758
Train loss:  214.7186737060547
_______________________


313it [00:34,  8.95it/s]
79it [00:08,  9.01it/s]


Epoch 10/15, Validation Accuracy: 0.4984
Validation MCC: 0.060048567424022696
Train loss:  215.183837890625
_______________________


235it [00:26,  8.92it/s]


KeyboardInterrupt: 

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
undir_classifier = Directed_classifier(input_size=2648, num_classes=2)
undir_classifier = undir_classifier.to(device)
model_undir = BertModel.from_pretrained('bert-base-uncased')
model_undir = model_undir.to(device)
criterion = nn.CrossEntropyLoss()
optimizer_undir = torch.optim.Adam(undir_classifier.parameters(), lr=1e-3)
num_epochs = 15
print("device: ", device)
train_directed(model_undir, optimizer_undir, criterion, undir_classifier, train_loader, val_loader, feat_train_loader, feat_test_loader, num_epochs, device)


device:  cuda


313it [00:35,  8.88it/s]
79it [00:08,  8.86it/s]


Epoch 1/15, Validation Accuracy: 0.5472
Validation MCC: 0.08202762169836987
Train loss:  1634.0604248046875
_______________________


313it [00:34,  8.99it/s]
79it [00:08,  9.05it/s]


Epoch 2/15, Validation Accuracy: 0.5596
Validation MCC: 0.10865824219315932
Train loss:  1025.471435546875
_______________________


...

Epoch 12/15, Validation Accuracy: 0.498
Validation MCC: 0.0758105258749771
Train loss:  801.6683349609375
_______________________


313it [00:34,  8.97it/s]
79it [00:08,  8.92it/s]


Epoch 13/15, Validation Accuracy: 0.5584
Validation MCC: 0.184983521852907
Train loss:  806.9306030273438
_______________________


282it [00:31,  8.94it/s]


KeyboardInterrupt: 

Accuracy has dropped(

#RFC

Lets train random forest classifier only on pre-calculated TDA features.

In [ ]:
rf_clf = RandomForestClassifier(n_estimators=100, random_state=42)
rf_clf.fit(undir_feat_train, y_train)

RandomForestClassifier(random_state=42)

In [ ]:
rf_clf.score(undir_feat_test, y_test)

0.6132

# XGBoost on precalculated features

In [ ]:
pip install xgboost

In [ ]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

model = xgb.XGBClassifier(objective='binary:logistic', random_state=42)
model.fit(undir_feat_train, y_train)
predictions = model.predict(undir_feat_test)
test_accuracy = accuracy_score(y_test, predictions)
print(f'Test Accuracy: {test_accuracy * 100:.2f}%')

Test Accuracy: 59.96%


# NN (baseline) + RFC ensemble (Weighted Voting)

Let's make an ensemble. We will use a baseline and RFС trained on TDA features

In [ ]:
def nn_probs(model, classifier, val_loader, device):
    classifier.eval()
    probs = []
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)[1]
            nn_probabilities = torch.softmax(classifier(outputs), dim=1).cpu().numpy()
            probs.extend(nn_probabilities)
    return probs

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_baseline = BertModel.from_pretrained('bert-base-uncased')
model_baseline = model_baseline.to(device)
nn_probabilities = nn_probs(model_baseline, classifier_baseline, val_loader, device)

In [ ]:
len(nn_probabilities)

2500

In [ ]:
np.save('nn_probs.npy', nn_probabilities)

In [ ]:
nn_probabilities_2d = np.vstack(nn_probabilities)

In [ ]:
nn_probabilities_2d

array([[0.8502205 , 0.14977948],
       [0.5684172 , 0.43158284],
       [0.63472414, 0.36527592],
       ...,
       [0.43231884, 0.5676812 ],
       [0.93980926, 0.06019071],
       [0.923689  , 0.07631107]], dtype=float32)

In [ ]:
rf_probabilities

array([[0.37, 0.63],
       [0.43, 0.57],
       [0.58, 0.42],
       ...,
       [0.54, 0.46],
       [0.64, 0.36],
       [0.45, 0.55]])

In [ ]:
alpha = 0.9

rf_probabilities = rf_clf.predict_proba(undir_feat_test)

# Ensemble: Weighted Voting
# Assign higher weight to the NN (0.7) and lower weight to RFC (0.3)
ensemble_probabilities = alpha * nn_probabilities_2d  + (1-alpha) * rf_probabilities
ensemble_predictions = ensemble_probabilities.argmax(axis=1)

# Evaluate the ensemble model
accuracy = accuracy_score(y_test, ensemble_predictions)
print(f'Ensemble Model Accuracy: {accuracy:.2f}')

Ensemble Model Accuracy: 0.68


In [ ]:
alphas = [0, 0.1, 0.2, 0.3, 0.4, 0.41, 0.42,0.43, 0.44,0.45, 0.46, 0.47, 0.48,0.49, 0.5, 0.51,0.52,0.53,0.56, 0.6, 0.7, 0.8, 0.9, 1]

rf_probabilities = rf_clf.predict_proba(undir_feat_test)

# Ensemble: Weighted Voting
for alpha in alphas:
    ensemble_probabilities = alpha * nn_probabilities_2d  + (1-alpha) * rf_probabilities
    ensemble_predictions = ensemble_probabilities.argmax(axis=1)
    accuracy = accuracy_score(y_test, ensemble_predictions)
    print("________")
    print("alpha:", alpha)
    print(f'Ensemble Model Accuracy: {accuracy:.2f}')

________
alpha: 0
Ensemble Model Accuracy: 0.61
________
alpha: 0.1
Ensemble Model Accuracy: 0.65
________
alpha: 0.2
Ensemble Model Accuracy: 0.68
________
alpha: 0.3
Ensemble Model Accuracy: 0.69
________
alpha: 0.4
Ensemble Model Accuracy: 0.70
________
alpha: 0.41
Ensemble Model Accuracy: 0.70
________
alpha: 0.42
Ensemble Model Accuracy: 0.70
________
alpha: 0.43
Ensemble Model Accuracy: 0.70
________
alpha: 0.44
Ensemble Model Accuracy: 0.69
________
alpha: 0.45
Ensemble Model Accuracy: 0.69
________
alpha: 0.46
Ensemble Model Accuracy: 0.70
________
alpha: 0.47
Ensemble Model Accuracy: 0.69
________
alpha: 0.48
Ensemble Model Accuracy: 0.70
________
alpha: 0.49
Ensemble Model Accuracy: 0.70
________
alpha: 0.5
Ensemble Model Accuracy: 0.70
________
alpha: 0.51
Ensemble Model Accuracy: 0.69
________
alpha: 0.52
Ensemble Model Accuracy: 0.69
________
alpha: 0.53
Ensemble Model Accuracy: 0.69
________
alpha: 0.56
Ensemble Model Accuracy: 0.69
________
alpha: 0.6
Ensemble Model Accu

As it can be seen with alpha = 0.5 our ensemble achieves accuracy 70%, which is 2% more than the baseline!!!